In [1]:
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
import torch 

from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import random
import torch
import torch.nn.functional as F
from torch import Tensor
from torch import nn
import random
import sklearn.metrics as m
import math
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.metrics import precision_recall_curve, auc
import re
from tqdm import tqdm
import os

In [2]:
buildings_df = pd.read_csv("/media/andrey/big/datasets/addr_spb/additional_data/building_20230808.csv")
df1 = pd.read_csv("/media/andrey/big/datasets/addr_spb/datasets/dataset_1.csv")
df2 = pd.read_csv("/media/andrey/big/datasets/addr_spb/datasets/dataset_2.csv")
df3 = pd.read_csv("/media/andrey/big/datasets/addr_spb/datasets/dataset_3.csv")
df4 = pd.read_csv("/media/andrey/big/datasets/addr_spb/datasets/dataset_4.csv")
df5 = pd.read_csv("/media/andrey/big/datasets/addr_spb/datasets/dataset_5.csv")

/tmp/ipykernel_16951/2785304975.py:1: DtypeWarning: Columns (4,6,7,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  buildings_df = pd.read_csv("/media/andrey/big/datasets/addr_spb/additional_data/building_20230808.csv")


In [3]:
buildings_df

,id,prefix_id,district_id,house,corpus,liter,villa,parcel,full_address,is_updated,is_actual,type,municipality_id,short_address,post_prefix,build_number
0,56343,11132,35,12,NaN,А,NaN,NaN,"город Пушкин, Кедринская улица, дом 12",True,False,NaN,107.0,"г.Пушкин, Кедринская ул., д. 12",NaN,NaN
1,595,6987,38,4Б,NaN,NaN,NaN,NaN,"поселок Ушково, Пляжевая улица, дом 4Б",True,False,NaN,128.0,"пос. Ушково, Пляжевая ул., д. 4Б",NaN,NaN
2,7134,6469,15,30,2,Е,NaN,NaN,"г.Санкт-Петербург, проспект Маршала Жукова, до...",True,False,Нежилое,30.0,"г.Санкт-Петербург, пр. Маршала Жукова, д. 30, ...",198303,NaN
3,124415,7838,38,5,2,А,NaN,NaN,"поселок Белоостров, Дюны, Центральная улица, д...",True,False,Нежилое,110.0,"пос. Белоостров, Дюны, Центральная ул., д. 5, ...",NaN,NaN
4,185368,4224,38,28,NaN,Б,NaN,NaN,"поселок Песочный, Речная улица, дом 28, литера Б",True,False,NaN,118.0,"пос. Песочный, Речная ул., д. 28, л. Б",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166640,18941,5289,32,3,NaN,В,NaN,NaN,"г.Санкт-Петербург, Бородинская улица, дом 3, л...",True,True,Нежилое,4.0,"г.Санкт-Петербург, Бородинская ул., д. 3, л. В",190000.0,NaN
166641,18942,5031,35,34/3,NaN,А,NaN,NaN,"посёлок Александровская, 2-я линия, дом 34/3, ...",True,True,Жилое,109.0,"пос. Александровская, 2-я линия, д. 34/3, л. А",196631.0,NaN
166642,18943,5623,38,39,NaN,NaN,NaN,NaN,"посёлок Ушково, Детская улица, дом 39",True,False,NaN,128.0,"пос. Ушково, Детская ул., д. 39",NaN,NaN
166643,18944,5787,34,25,NaN,Б,NaN,NaN,"г.Санкт-Петербург, Заусадебная улица, дом 25, ...",True,False,Нежилое,68.0,"г.Санкт-Петербург, Заусадебная ул., д. 25, л. Б",NaN,NaN


In [4]:
df_train = pd.concat([df1, df2, df3, df5])
df_test = df4

In [5]:
df_train = df_train.dropna(subset=['target_building_id'])
df_train['target_building_id'] = df_train['target_building_id'].astype('int')

In [6]:
df_train

,Unnamed: 0,address,target_building_id,target_address
0,4070,"196641, Санкт-Петербург г, Школьная ул, д.5",76337,"Санкт-Петербург, город Павловск, Пязелево, Шко..."
1,7573,"Санкт-Петербург (г.), Железнодорожный (пр-кт.)...",220011,"г.Санкт-Петербург, Железнодорожный проспект, д..."
2,5815,"198325, Санкт-Петербург г, Красное Село г, Теа...",109979,"Санкт-Петербург, город Красное Село, Дудергоф,..."
3,2159,"195256, Санкт-Петербург (г.), Верности (ул.), ...",72112,"г.Санкт-Петербург, улица Верности, дом 28, кор..."
4,1476,"194291, Санкт-Петербург (г.), Кустодиева (ул.)...",186531,"г.Санкт-Петербург, улица Кустодиева, дом 1, ли..."
...,...,...,...,...
138283,118130,"г Санкт-Петербург, тер Сергиево, ул Дмитриевская",80244,"муниципальный округ Константиновское, территор..."
138284,125799,"Россия, Санкт-Петербург, Кирочная улица, 28",93144,"г.Санкт-Петербург, Кирочная улица, дом 28, лит..."
138285,14324,"Красногвардейский район, муниципальный округ П...",98201,"г.Санкт-Петербург, Ленская улица, дом 6, корпу..."
138286,128490,"Россия, Санкт-Петербург, посёлок Парголово, ул...",15911,"посёлок Парголово, проспект Энгельса, дом 162,..."


In [7]:
len(df_train["target_building_id"].unique())

33857

In [8]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased").to(device)
sentences = ["Hello World", "Привет Мир"]
encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=64, return_tensors='pt').to(device)
with torch.no_grad():
    model_output = model(**encoded_input)
embeddings = model_output.pooler_output
embeddings = torch.nn.functional.normalize(embeddings)
print(embeddings, embeddings.shape)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[ 0.0206, -0.0072,  0.0213,  ..., -0.0186,  0.0212,  0.0166],
        [ 0.0181, -0.0124,  0.0247,  ..., -0.0321,  0.0258,  0.0173]],
       device='cuda:0') torch.Size([2, 768])


In [9]:
HARD_SAMPLES = {}
def add_samples(building_ids):
    if(len(building_ids) < 2): return
    if(len(building_ids) > 200): return
    for building_id_1 in building_ids:
        for building_id_2 in building_ids:
            if building_id_1 != building_id_2:
                if building_id_1 not in HARD_SAMPLES: HARD_SAMPLES[building_id_1] = set()
                HARD_SAMPLES[building_id_1].add(building_id_2)
                    
for building_ids in buildings_df.groupby('prefix_id')['id'].apply(list).to_dict().values():
    add_samples(building_ids)
for building_ids in buildings_df.groupby('district_id')['id'].apply(list).to_dict().values():
    add_samples(building_ids)
for building_ids in buildings_df.groupby('house')['id'].apply(list).to_dict().values():
    add_samples(building_ids)
        

In [10]:
BUILDINGS_DATA_AVG_EMB = np.zeros((len(buildings_df["id"]), 768))
BUILDINGS_DATA_EMB = [] #np.zeros((len(buildings_df["id"]), 30, 768))
BUILDINGS_DATA_ID = []
BUILDINGS_DATA_ADDRESS = []
BUILDINGS_DATA_MAPPING = {}

In [11]:
BUILDINGS_DATA = {}
for i, (building_id, building_address) in tqdm(enumerate(zip(buildings_df["id"], buildings_df["full_address"])), total=len(buildings_df["id"])):
    encoded_input = tokenizer([building_address], padding=True, truncation=True, max_length=256, return_tensors='pt').to(device)
    with torch.no_grad():
        model_output = model(**encoded_input)
    embeddings = model_output.last_hidden_state
#     embeddings = torch.zeros((1, 1, 768))

    avg_emb = torch.nn.functional.normalize(embeddings[0].mean(dim=0), dim=-1).cpu().numpy()
    embeddings = embeddings[0].cpu().numpy() #torch.nn.functional.normalize(embeddings, dim=-1)[0].cpu().numpy()
    BUILDINGS_DATA[building_id] = (building_id, building_address, embeddings)
    
    BUILDINGS_DATA_AVG_EMB[i] = avg_emb
    BUILDINGS_DATA_EMB.append(embeddings)
    BUILDINGS_DATA_ID.append(building_id)
    BUILDINGS_DATA_ADDRESS.append(building_address)
    BUILDINGS_DATA_MAPPING[building_id] = i

100%|██████████████████████████████████| 166645/166645 [16:48<00:00, 165.29it/s]


In [24]:
BUILDINGS_DATA_ID = np.array(BUILDINGS_DATA_ID)
BUILDINGS_DATA_ADDRESS = np.array(BUILDINGS_DATA_ADDRESS)

In [12]:
#for i in range(0, BUILDINGS_DATA_EMB.shape[0]):
#    current_emb = BUILDINGS_DATA_EMB[[i]]
#    dists = ((current_emb - BUILDINGS_DATA_EMB) ** 2).sum(dim=-1)
#    print(a)

In [13]:
TRAIN_DATA = []
for building_id, building_address in tqdm(zip(df_train["target_building_id"], df_train["address"]), total=len(df_train["target_building_id"])):
    encoded_input = tokenizer([building_address], padding=True, truncation=True, max_length=256, return_tensors='pt').to(device)
    with torch.no_grad():
        model_output = model(**encoded_input)
    embeddings = model_output.last_hidden_state
#     embeddings = torch.zeros((1, 1, 768))
    embeddings = embeddings[0].cpu().numpy() #torch.nn.functional.normalize(embeddings, dim=-1)[0].cpu().numpy()
    TRAIN_DATA.append((building_id, embeddings))

100%|██████████████████████████████████| 149022/149022 [14:18<00:00, 173.62it/s]


In [14]:
TEST_EMBS = []
for address in tqdm(df_test["address"]):
    encoded_input = tokenizer([address], padding=True, truncation=True, max_length=256, return_tensors='pt').to(device)
    with torch.no_grad():
        model_output = model(**encoded_input)
    embeddings = model_output.last_hidden_state
#     embeddings = torch.zeros((1, 1, 768))
    embeddings = embeddings[0].cpu().numpy() #torch.nn.functional.normalize(embeddings, dim=-1)[0].cpu().numpy()
    TEST_EMBS.append(embeddings)

100%|████████████████████████████████████| 25268/25268 [02:28<00:00, 170.59it/s]


In [15]:
def trunc_or_pad(arr):
    if arr.shape[0] >40: return arr[:40]
    else:
        new_arr = np.zeros((40, 768))
        new_arr[:arr.shape[0]] = arr
        return new_arr

In [16]:
def get_mask(arr):
    mask = np.zeros((40, 1))
    mask[:max(arr.shape[0], 40)] = 1.
    return mask

In [18]:
class TrainDataset(torch.utils.data.Dataset):

    def __init__(self, train_data, building_data_mapping, building_data_emb, hard_samples):
        self.train_data = train_data
        self.building_data_mapping = building_data_mapping
        self.building_data_emb = building_data_emb
        self.hard_samples = hard_samples

    def __getitem__(self, index):
        building_id, input_embs = self.train_data[index]
        index_pos = self.building_data_mapping[building_id]
        positive_build_emb = self.building_data_emb[index_pos]
        
        if building_id in self.hard_samples and random.randint(0, 100) > 50:
            neg_building_id = random.choice(list(self.hard_samples[building_id]))
            assert neg_building_id != building_id
            index_neg = self.building_data_mapping[neg_building_id]
        else:
            index_neg = index_pos
            while index_neg == index_pos: index_neg = random.randint(0, len(self.building_data_emb) - 1) 
        negative_build_emb = self.building_data_emb[index_neg]
        
        return  trunc_or_pad(input_embs), trunc_or_pad(positive_build_emb), trunc_or_pad(negative_build_emb),get_mask(input_embs), get_mask(positive_build_emb), get_mask(negative_build_emb) 
        

    def __len__(self):
        return len(self.train_data)

In [19]:
class Model(nn.Module):
    def __init__(self,):
        super().__init__()
 
        self.pre_net = nn.Sequential(
            nn.Linear(768, 512),
            nn.LeakyReLU(),
            nn.Linear(512, 512)
        )
        self.norm1 = nn.Linear(512, 512)
        self.attn1 = nn.Linear(512, 512)
    
    
        self.postnet = nn.Sequential(
            nn.Linear(512, 512),
            nn.LeakyReLU(),
            nn.Linear(512, 256)
        )
    
    def process_vec(self, a, mask):
        a = self.pre_net(a)
        normalized = self.norm1(a)
        attn = torch.sigmoid(self.attn1(a))
        a = (normalized * attn * mask).sum(dim=-2) / ((attn * mask).sum(dim=-2) + 1e-8)
        return a
    
    def forward(self, a, mask):
        a = self.process_vec(a, mask)
        res = self.postnet(a)
        return res
#         return torch.sigmoid(res.squeeze(-1))

In [20]:
train_dataset = TrainDataset(TRAIN_DATA, BUILDINGS_DATA_MAPPING, BUILDINGS_DATA_EMB, HARD_SAMPLES)
train_set = DataLoader(train_dataset, batch_size=512, num_workers=4, shuffle=True)

In [21]:
adress_model = Model().cuda()
optim_model = torch.optim.Adam(adress_model.parameters(), 1e-4, weight_decay=1e-6)

In [22]:
%env TOKENIZERS_PARALLELISM=false

env: TOKENIZERS_PARALLELISM=false


In [33]:
best_score = 0
print("TRAINING..")
triplet_loss = nn.TripletMarginLoss(margin=1.0, p=2)
for epoch in range(1000):
    print("Epoch:", epoch)
    model.train()
    losses = []
    for i, batch in enumerate(train_set):

        input_embs, positive_build_emb, negative_build_emb, input_embs_mask, positive_build_emb_mask, negative_build_emb_mask  = batch
        input_embs = input_embs.float().cuda()
        positive_build_emb = positive_build_emb.float().cuda()
        negative_build_emb = negative_build_emb.float().cuda()

        input_embs_mask = input_embs_mask.float().cuda()
        positive_build_emb_mask = positive_build_emb_mask.float().cuda()
        negative_build_emb_mask = negative_build_emb_mask.float().cuda()
        
        res_input = adress_model(input_embs, input_embs_mask)
        res_pos = adress_model(positive_build_emb, positive_build_emb_mask)
        res_neg = adress_model(negative_build_emb, negative_build_emb_mask)
        
        loss = triplet_loss(res_input, res_pos, res_neg)
        
#         loss_pos = torch.nn.functional.binary_cross_entropy(res_pos, torch.ones_like(res_pos))
#         loss_neg = torch.nn.functional.binary_cross_entropy(res_neg, torch.ones_like(res_neg))

#         loss = loss_pos + loss_neg
            
        optim_model.zero_grad()
        loss.backward()
        optim_model.step()
        losses.append(loss.item())

        if i % 100 ==0:
            print(i,"OF", len(train_set), "Loss:", loss.item())
        #break
    
    if epoch%1==0:
        adress_model.eval()
        with torch.no_grad():
            FINAL_build_embs = []
            BATCH_SIZE = 2024
            for i in range(0, len(BUILDINGS_DATA_EMB), BATCH_SIZE):
                    embs = BUILDINGS_DATA_EMB[i:i+BATCH_SIZE]
                    embs = np.array([trunc_or_pad(e) for e in embs])
                    msks = np.array([get_mask(e) for e in embs])
                    
                    embs = torch.from_numpy(embs).float().cuda()
                    msks = torch.from_numpy(msks).float().cuda()
                    res_pos = adress_model(embs, msks)
                    FINAL_build_embs.append(res_pos)
            FINAL_build_embs = torch.cat(FINAL_build_embs, dim=0)

            scores = []
            for input_address, target_address, ie in tqdm(zip(df_test["address"], df_test["target_address"], TEST_EMBS)):

                input_emb = torch.from_numpy(trunc_or_pad(ie)).unsqueeze(0).float().cuda()
                input_emb_msk = torch.from_numpy(get_mask(ie)).unsqueeze(0).float().cuda()
                FINAL_input_emb = adress_model(input_emb, input_emb_msk)
                res = ((FINAL_input_emb - FINAL_build_embs)**2).mean(dim=-1)
                topkres = torch.topk(-res, 10)
                topindexes = topkres.indices.cpu().numpy()
                top_buildings_addresses = [BUILDINGS_DATA_ADDRESS[ti] for ti in topindexes]
                result_score = 0
                for addr in top_buildings_addresses:
                    if addr == target_address:
                        result_score = 1
                scores.append(result_score)
            scores = np.array(scores)
            print("SCORE:", scores.mean())
            if scores.mean() > best_score:
                best_score = scores.mean()
                print("SAVE BEST...")
                torch.save(adress_model.state_dict(), "address_model.trc")
                np.save("BUILDINGS_DATA_ADDRESS.npy", BUILDINGS_DATA_ADDRESS, allow_pickle=True)
                np.save("BUILDINGS_DATA_ID.npy", BUILDINGS_DATA_ID, allow_pickle=True)
                np.save("BUILDINGS_DATA_EMB.npy", FINAL_build_embs.detach().cpu().numpy(), allow_pickle=True)
                

TRAINING..
Epoch: 0
0 OF 292 Loss: 0.03975021094083786
100 OF 292 Loss: 0.06475387513637543
200 OF 292 Loss: 0.02913709729909897


25268it [01:34, 267.66it/s]


SCORE: 0.1410083900585721
SAVE BEST...
Epoch: 1
0 OF 292 Loss: 0.03185538947582245


KeyboardInterrupt: 

In [37]:
!git clone https://huggingface.co/DeepPavlov/rubert-base-cased

Клонирование в «rubert-base-cased»…
remote: Enumerating objects: 41, done.
remote: Total 41 (delta 0), reused 0 (delta 0), pack-reused 41
Распаковка объектов: 100% (41/41), готово.
Фильтруется содержимое: 100% (2/2), 1.33 GiB | 57.73 MiB/s, готово.


In [38]:
tokenizer22 = AutoTokenizer.from_pretrained('./rubert-base-cased')
model22 = AutoModel.from_pretrained("./rubert-base-cased")

Some weights of the model checkpoint at ./rubert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
